In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import DataLoader
from torchvision.datasets import CIFAR10, MNIST
from torchvision.transforms import Compose, ToTensor, Normalize, RandomHorizontalFlip, RandomRotation

from cr_ibp_modules import *
from cr_ibp_utils import *

import numpy as np
import matplotlib.pyplot as plt
import matplotlib as mpl
from tqdm import trange
from copy import deepcopy
    
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
torch.manual_seed(42);

In [3]:
dataset = MNIST('../data/mnist',
                download=True,
                train=True,
                transform=ToTensor())
BATCH_SIZE = 32
IMAGE_SIZE = dataset[0][0].shape[1:]
FC_DIM = np.prod(IMAGE_SIZE)
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [50000, 10000])
dataloader_train = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
dataloader_val = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=True)
dataloader_test = DataLoader(dataset, batch_size=BATCH_SIZE, shuffle=False)
dataloader_vis = DataLoader(dataset, batch_size=1, shuffle=False)

In [6]:
model_cr = Net2()
opt = torch.optim.Adam(model_cr.parameters())
lossf = nn.CrossEntropyLoss()

In [7]:
epochs = 20
epochs_burnin = 5
lr_schedule = np.flip(np.linspace(1e-6, 1e-4, epochs))
h_schedule = np.linspace(0, 1.5, epochs_burnin).tolist() + np.repeat([1.5], epochs-epochs_burnin).tolist()
cr_weight = 4

In [ ]:
train(model_cr, dataloader_train, dataloader_val, lossf, opt,
      lr_schedule=lr_schedule, h_schedule=h_schedule, mode='cr', epochs=20)

/Users/nickstulov/Desktop/Study/dl/Project/cr_ibp_utils.py:57: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  preds = np.argmax(F.softmax(logits).detach().numpy(), axis=-1)


In [ ]:
torch.save(model, '../models/model_cr_mnist.mdl')